# basic tests

In [1]:
from PandaStation import PandaStation
from PandaStation import AddShape

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydot
import pydrake.all
import os
from IPython.display import display, SVG



import pydrake.all
from pydrake.geometry import Cylinder, Box
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role
    )
from PandaInverseKinematics import PandaInverseKinematics, PandaIKTraj, Waypoint, Trajectory

In [2]:
builder = pydrake.systems.framework.DiagramBuilder()
station = builder.AddSystem(PandaStation())
station.SetupDefaultStation()
station.Finalize()
diagram = builder.Build()


file = open("./media/pictures/diagram.svg", "w")
#file.write(GenerateHtml(diagram))

file.write(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg().decode("utf-8"))
file.close()
#pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].write_html("./media/pictures/panda_station.html")

In [3]:
def TestWithMeshcat():
    builder = pydrake.systems.framework.DiagramBuilder()
    station = builder.AddSystem(PandaStation())
    station.SetupDefaultStation()
    station.Finalize()

    visualizer = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(
        builder, output_port=station.GetOutputPort("geometry_query"), zmq_url=zmq_url)

    diagram = builder.Build()

    context = diagram.CreateDefaultContext()
    visualizer.load(visualizer.GetMyContextFromRoot(context))
    diagram.Publish(context)

#TestWithMeshcat()

# Testing basic control

In [4]:
# We can write a new System by deriving from the LeafSystem class.
# There is a little bit of boiler plate, but hopefully this example makes sense.
class PseudoInverseController(pydrake.systems.framework.LeafSystem):
    def __init__(self, plant):
        pydrake.systems.framework.LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("panda")
        self._G = plant.GetBodyByName("panda_hand").body_frame()
        self._W = plant.world_frame()

        self.DeclareVectorInputPort("panda_position", pydrake.systems.framework.BasicVector(7))
        self.DeclareVectorOutputPort("panda_velocity", pydrake.systems.framework.BasicVector(7), 
                                     self.CalcOutput)

    def CalcOutput(self, context, output):
        q = self.get_input_port().Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, pydrake.multibody.tree.JacobianWrtVariable.kQDot, 
            self._G, [0,0,0], self._W, self._W)
        J_G = J_G[:,0:7] # Ignore gripper terms
        
        V_G_desired = np.array([0,    # rotation about x
                                -.1,  # rotation about y
                                0,    # rotation about z
                                0,    # x
                                -.05, # y
                                -.1]) # z
        v = np.linalg.pinv(J_G).dot(V_G_desired)
        output.SetFromVector(v)

        
def jacobian_controller_example():
    builder = pydrake.systems.framework.DiagramBuilder()

    station = builder.AddSystem(PandaStation())
    station.SetupDefaultStation()
    station.Finalize()

    controller = builder.AddSystem(PseudoInverseController(
        station.get_multibody_plant()))
    integrator = builder.AddSystem(pydrake.systems.primitives.Integrator(7))

    builder.Connect(controller.get_output_port(), 
                    integrator.get_input_port())
    builder.Connect(integrator.get_output_port(),
                    station.GetInputPort("panda_position"))
    builder.Connect(station.GetOutputPort("panda_position_measured"),
                    controller.get_input_port())

    meshcat =pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
        station.get_scene_graph(),
        output_port=station.GetOutputPort("query_object"),
        zmq_url=zmq_url)

    diagram = builder.Build()
    simulator = pydrake.systems.analysis.Simulator(diagram)
    station_context = station.GetMyContextFromRoot(simulator.get_mutable_context())
    #station.GetInputPort("panda_feedforward_torque").FixValue(station_context, np.zeros((7,1))) // this will already be 0
    station.GetInputPort("hand_position").FixValue(station_context, [0.08]) # taking the desired hand separation
    # TODO(russt): Add this missing python binding
    #integrator.set_integral_value(
    #    integrator.GetMyContextFromRoot(simulator.get_mutable_context()), 
    #        station.GetIiwaPosition(station_context))
    integrator.GetMyContextFromRoot(
        simulator.get_mutable_context()).get_mutable_continuous_state_vector().SetFromVector(
        station.GetPandaPosition(station_context))

    simulator.set_target_realtime_rate(1.0)
    simulator.AdvanceTo(0.01);

    return simulator

#simulator = jacobian_controller_example()

In [5]:
#simulator.AdvanceTo(5.0)

# testing pick and place

### create controller

In [6]:
class PseudoInverseController(pydrake.systems.framework.LeafSystem):
    def __init__(self, plant):
        pydrake.systems.framework.LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._panda = plant.GetModelInstanceByName("panda")
        self._G = plant.GetBodyByName("panda_hand").body_frame()
        self._W = plant.world_frame()

        self.w_G_port = self.DeclareVectorInputPort("omega_WG", pydrake.systems.framework.BasicVector(3))
        self.v_G_port = self.DeclareVectorInputPort("v_WG", pydrake.systems.framework.BasicVector(3))
        self.q_port = self.DeclareVectorInputPort("panda_position", pydrake.systems.framework.BasicVector(7))
        self.DeclareVectorOutputPort("panda_velocity", pydrake.systems.framework.BasicVector(7), 
                                     self.CalcOutput)

        self.panda_start = plant.GetJointByName("panda_joint1").velocity_start()
        self.panda_end = plant.GetJointByName("panda_joint7").velocity_start()

    def CalcOutput(self, context, output):
        w_G = self.w_G_port.Eval(context)
        v_G = self.v_G_port.Eval(context)
        V_G = np.hstack([w_G, v_G])
        q = self.q_port.Eval(context)
        self._plant.SetPositions(self._plant_context, self._panda, q)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, pydrake.multibody.tree.JacobianWrtVariable.kV, 
            self._G, [0,0,0], self._W, self._W)
        J_G = J_G[:,self.panda_start:self.panda_end+1] # Only iiwa terms.
        v = np.linalg.pinv(J_G).dot(V_G)
        output.SetFromVector(v)

### Create gripper frames

In [7]:
def make_gripper_frames(X_G, X_O):
  """
  Takes a partial specification with X_G["initial"] and X_O["initial"] and X_0["goal"], and 
  returns a X_G and times with all of the pick and place frames populated.
  """
  # Define (again) the gripper pose relative to the object when in grasp.
  p_GgraspO = [0, 0, 0.12]
  R_GgraspO = RotationMatrix.MakeXRotation(np.pi)
  X_GgraspO = RigidTransform(R_GgraspO, p_GgraspO)
  X_OGgrasp = X_GgraspO.inverse()
  # pregrasp is negative y in the gripper frame (see the figure!).
  X_GgraspGpregrasp = RigidTransform([0, 0, -0.08])

  X_G["pick"] = X_O["initial"].multiply(X_OGgrasp)
  X_G["prepick"] = X_G["pick"].multiply(X_GgraspGpregrasp)
  X_G["place"] = X_O["goal"].multiply(X_OGgrasp)
  X_G["preplace"] = X_G["place"].multiply(X_GgraspGpregrasp)

  # I'll interpolate a ahlfway orientation by converting to axis angle and halving the angle.
  X_GprepickGpreplace = X_G["prepick"].inverse().multiply(X_G["preplace"])
  angle_axis = X_GprepickGpreplace.rotation().ToAngleAxis()
  X_GprepickGclearance = RigidTransform(AngleAxis(angle=angle_axis.angle()/2.0, axis=angle_axis.axis()), 
                                        X_GprepickGpreplace.translation()/2.0 + np.array([0, 0, -0.4]))
  X_G["clearance"] = X_G["prepick"].multiply(X_GprepickGclearance)

  # Now let's set the timing
  times = {"initial": 0}
  X_GinitialGprepick = X_G["initial"].inverse().multiply(X_G["prepick"])
  times["prepick"] = times["initial"] + 10.0*np.linalg.norm(X_GinitialGprepick.translation())
  # Allow some time for the gripper to close.
  times["pick_start"] = times["prepick"] + 2.0
  times["pick_end"] = times["pick_start"] + 2.0
  times["postpick"] = times["pick_end"] + 2.0
  time_to_from_clearance = 10.0*np.linalg.norm(X_GprepickGclearance.translation())
  times["clearance"] = times["postpick"] + time_to_from_clearance
  times["preplace"] = times["clearance"] + time_to_from_clearance
  times["place_start"] = times["preplace"] + 2.0
  times["place_end"] = times["place_start"] + 2.0
  times["postplace"] = times["place_end"] + 2.0

  return X_G, times

X_G = {"initial": RigidTransform(RotationMatrix.MakeXRotation(-np.pi), [0, -0.25, 0.25])}
X_O = {"initial": RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [-.2, -.75, 0.025]),
       "goal": RigidTransform(RotationMatrix.MakeZRotation(np.pi),[.75, 0, 0.025])}
X_G, times = make_gripper_frames(X_G, X_O)
print(f"Sanity check: The entire maneuver will take {times['postplace']} seconds to execute.")

def make_gripper_position_trajectory(X_G, times):
    """
    Constructs a gripper position trajectory from the plan "sketch".
    """

    traj = pydrake.trajectories.PiecewisePolynomial.FirstOrderHold(
        [times["initial"], times["prepick"]], np.vstack([X_G["initial"].translation(), X_G["prepick"].translation()]).T)

    
    traj.AppendFirstOrderSegment(times["pick_end"], X_G["pick"].translation())
    traj.AppendFirstOrderSegment(times["postpick"], X_G["prepick"].translation())
    traj.AppendFirstOrderSegment(times["clearance"], X_G["clearance"].translation())
    traj.AppendFirstOrderSegment(times["preplace"], X_G["preplace"].translation())
    traj.AppendFirstOrderSegment(times["place_start"], X_G["place"].translation())
    traj.AppendFirstOrderSegment(times["place_end"], X_G["place"].translation())
    traj.AppendFirstOrderSegment(times["postplace"], X_G["preplace"].translation())

    return traj

traj_p_G = make_gripper_position_trajectory(X_G, times)

def make_gripper_orientation_trajectory(X_G, times):
    """
    Constructs a gripper position trajectory from the plan "sketch".
    """
    traj = pydrake.trajectories.PiecewiseQuaternionSlerp();
    traj.Append(times["initial"], X_G["initial"].rotation())
    traj.Append(times["prepick"], X_G["prepick"].rotation())
    traj.Append(times["pick_start"], X_G["pick"].rotation())
    traj.Append(times["pick_end"], X_G["pick"].rotation())
    traj.Append(times["postpick"], X_G["prepick"].rotation())
    traj.Append(times["clearance"], X_G["clearance"].rotation())
    traj.Append(times["preplace"], X_G["preplace"].rotation())
    traj.Append(times["place_start"], X_G["place"].rotation())
    traj.Append(times["place_end"], X_G["place"].rotation())
    traj.Append(times["postplace"], X_G["preplace"].rotation())

    return traj

traj_R_G = make_gripper_orientation_trajectory(X_G, times)

opened = np.array([0.08]);
closed = np.array([0.0]);

def make_hand_command_trajectory(times):
    traj_hand_command = pydrake.trajectories.PiecewisePolynomial.FirstOrderHold(
        [times["initial"], times["pick_start"]], np.hstack([[opened], [opened]]))
    traj_hand_command.AppendFirstOrderSegment(times["pick_end"], closed)
    traj_hand_command.AppendFirstOrderSegment(times["place_start"], closed)
    traj_hand_command.AppendFirstOrderSegment(times["place_end"], opened)
    traj_hand_command.AppendFirstOrderSegment(times["postplace"], opened)
    return traj_hand_command

#traj_hand_command = make_hand_command_trajectory(times)

Sanity check: The entire maneuver will take 31.899582791758867 seconds to execute.


### testing pick and place

In [8]:
X_O = {"initial": RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [-.2, -.65, 0.09]),
       "goal": RigidTransform(RotationMatrix.MakeZRotation(np.pi),[.5, 0, 0.09])}

builder = pydrake.systems.framework.DiagramBuilder()

station = builder.AddSystem(PandaStation())
station.SetupBinStation()
station.AddManipulandFromFile(
    "drake/examples/manipulation_station/models/061_foam_brick.sdf",
    X_O["initial"])

station.AddManipulandFromFile(
    "drake/examples/manipulation_station/models/061_foam_brick.sdf",
    RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [-.1, -.65, 0.09]))

station.AddManipulandFromFile(
    "drake/examples/manipulation_station/models/061_foam_brick.sdf",
    RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [-.3, -.65, 0.09]))

plant = station.get_multibody_plant()

station.Finalize()

# Find the initial pose of the gripper (as set in the default Context)
station_context = station.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(station_context)
X_G = {"initial": plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("panda_hand"))}
X_G, times = make_gripper_frames(X_G, X_O)
print(f"Sanity check: The entire maneuver will take {times['postplace']} seconds to execute.")

# Make the trajectories
traj_p_G = make_gripper_position_trajectory(X_G, times)
traj_v_G = traj_p_G.MakeDerivative()
traj_R_G = make_gripper_orientation_trajectory(X_G, times)
traj_w_G = traj_R_G.MakeDerivative()

v_G_source = builder.AddSystem(pydrake.systems.primitives.TrajectorySource(traj_v_G))
v_G_source.set_name("v_WG")
w_G_source = builder.AddSystem(pydrake.systems.primitives.TrajectorySource(traj_w_G))
w_G_source.set_name("omega_WG")
controller = builder.AddSystem(PseudoInverseController(plant))
controller.set_name("PseudoInverseController")
builder.Connect(v_G_source.get_output_port(), controller.GetInputPort("v_WG"))
builder.Connect(w_G_source.get_output_port(), controller.GetInputPort("omega_WG"))

integrator = builder.AddSystem(pydrake.systems.primitives.Integrator(7))
integrator.set_name("integrator")
builder.Connect(controller.get_output_port(), 
                integrator.get_input_port())
builder.Connect(integrator.get_output_port(),
                station.GetInputPort("panda_position"))
builder.Connect(station.GetOutputPort("panda_position_measured"),
                controller.GetInputPort("panda_position"))

traj_hand_command = make_hand_command_trajectory(times)
hand_source = builder.AddSystem(pydrake.systems.primitives.TrajectorySource(traj_hand_command))
hand_source.set_name("hand_command")
builder.Connect(hand_source.get_output_port(), station.GetInputPort("hand_position"))

meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
    station.get_scene_graph(),
    output_port=station.GetOutputPort("query_object"),
    zmq_url=zmq_url
)

diagram = builder.Build()
diagram.set_name("pick_and_place")
diagram_context = diagram.CreateDefaultContext()

simulator = pydrake.systems.analysis.Simulator(diagram, diagram_context)
station_context = station.GetMyContextFromRoot(simulator.get_mutable_context())

integrator.GetMyContextFromRoot(simulator.get_mutable_context()).get_mutable_continuous_state_vector().SetFromVector(station.GetPandaPosition(station_context))

#simulator.set_target_realtime_rate(1.0)
#meshcat.start_recording()
#simulator.AdvanceTo(traj_p_G.end_time())
#meshcat.stop_recording()
#meshcat.publish_recording()

Sanity check: The entire maneuver will take 35.567723750562024 seconds to execute.
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6006...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7006/static/
Connected to meshcat-server.


In [9]:
#file = open("basic_pick_and_place.html", "w")
#file.write(meshcat.vis.static_html())
#file.close()

# Testing Pick and Place Optimization Problem


### Create IK Controller

In [10]:
def CreatePandaControllerPlant():
    """creates plant that includes only the robot, gripper, and collidable objects, used for controllers."""
    robot_sdf_path = FindResourceOrThrow(
        "drake/manipulation/models/franka_description/urdf/panda_arm.urdf")
    gripper_sdf_path = FindResourceOrThrow(
        "drake/manipulation/models/franka_description/urdf/hand.urdf")
    
    
    sim_timestep = 1e-3
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    parser = Parser(plant)
    
    panda = parser.AddModelFromFile(robot_sdf_path, "panda")
    parser.AddModelFromFile(gripper_sdf_path, "hand")
    

    #joints = plant.GetJointIndices(panda)
    #plant.get_mutable_joint(joints[0]).set_default_angle(np.pi/2)
    
    c1 = AddShape(plant, Cylinder(0.04, 0.6), "c1")
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("c1", c1), RigidTransform([0.15, -0.15, 0.3])) # add a model to test collisions

    plant.WeldFrames(
        frame_on_parent_P=plant.world_frame(),
        frame_on_child_C =plant.GetFrameByName("panda_link0"),
        X_PC = RigidTransform(RollPitchYaw(0,0,0), np.array([0,0,0])))
    plant.WeldFrames(
        frame_on_parent_P=plant.GetFrameByName("panda_link8"),
        frame_on_child_C=plant.GetFrameByName("panda_hand"),
        X_PC=RigidTransform(RollPitchYaw(0, 0, 0), np.array([0, 0, 0]))
    )
    plant.mutable_gravity_field().set_gravity_vector([0, 0, 0])
    plant.Finalize()
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    return plant, plant_context

In [11]:
def AddCollisionConstraints(ik, min_distance, arm_geometries, collidable_geometries):
    for i in range(len(arm_geometries)):
        for j in range(len(collidable_geometries)):
            pair = (arm_geometries[i], collidable_geometries[j])
            ik.AddDistanceConstraint(pair, min_distance, 1000) # we dont care about the upper bound so set it to a large number


def create_q_knots(pose_lst, h, panda, plant, plant_context, to_avoid):
    
    #plant, plant_context = CreatePandaControllerPlant()
    '''
    arm_geometries = []
    bodies = plant.GetBodyIndices(plant.GetModelInstanceByName("panda"))
    for i in bodies:
        b = plant.get_body(i)
        if (b.name() == "panda_link0"): # this is because the distance constraint is not supported between two static objects in the scene
            continue
        arm_geometries+=plant.GetCollisionGeometriesForBody(b)
        
    collidable_names = ["c1"]
    to_avoid_geometries = []
    for name in collidable_names:
        bodies = plant.GetBodyIndices(plant.GetModelInstanceByName(name))
        for i in bodies:
            to_avoid_geometries+=plant.GetCollisionGeometriesForBody(plant.get_body(i))
    '''
    trans_tol = np.ones(3)*0.001
    theta_tol = 0
    
    q_knots = []
    
    W = plant.world_frame()
    G = plant.GetFrameByName("panda_hand") # gripper frame
    print(plant.num_positions())
    #print(plant_context)
    #q_nominal = np.array([ 0., 0.6, 0., -1.75, 0., 1., 0., 0, 0]) # nominal joint for joint-centering.
    q_nominal = np.array([ 0., 0.6, 0., -1.75, 0., 1., 0.]) # nominal joint for joint-centering.
    q_vel_limit = np.array([150.0, 150.0, 150.0, 150.0, 180.0, 180.0, 180.0]) # in deg/s
    q_vel_limit = q_vel_limit*(np.pi/180.0)

    
    for i in range(len(pose_lst)):
        des_pose = pose_lst[i]
        
        p_WG = des_pose.translation()
        R_WG = des_pose.rotation()
        ik = PandaInverseKinematics(plant, plant_context, panda, avoid_names = to_avoid)
        ik.AddPositionConstraint(p_WG - trans_tol, p_WG + trans_tol)
        ik.AddOrientationConstraint(R_WG, 0.01)
        ik.AddMinDistanceConstraint(0.001)
        prog = ik.get_prog()
        q = ik.get_q()
        
        '''
        ik = InverseKinematics(plant, plant_context)
        q = ik.q()
        
        prog = ik.prog()
        
        constraint1 = ik.AddPositionConstraint(frameA = W, 
                                 frameB = G, 
                                 p_BQ = np.zeros(3), 
                                 p_AQ_lower = p_WG-trans_tol, 
                                 p_AQ_upper = p_WG+trans_tol)
        constraint2 = ik.AddOrientationConstraint(frameAbar = W,
                                    R_AbarA = R_WG,
                                    frameBbar = G,
                                    R_BbarB = RotationMatrix(),
                                    theta_bound = theta_tol)
        '''
        prog.AddQuadraticErrorCost(np.identity(len(q)), q_nominal, q) # joint centering constraint
        #AddCollisionConstraints(ik, 0.01,arm_geometries, to_avoid_geometries)

        if len(q_knots) == 0:      
            prog.SetInitialGuess(q, q_nominal)
        else:
            for i in range(len(q_vel_limit)):
                #prog.AddConstraint((q[i] - q_knots[-1][i])/h <= q_vel_limit[i]) # joint velocity constraint
                prog.AddQuadraticErrorCost(np.identity(len(q)), q_knots[-1], q)
            prog.SetInitialGuess(q, q_knots[-1])
        result = Solve(prog)
        if not result.is_success():
            print("failed to find IK solution")
        q_knots.append(result.GetSolution(q))
        
    return q_knots

### Create Trajectories and Test

In [12]:
'''
X_O = {"initial": RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [-.1, -.65, 0.09]),
       "goal": RigidTransform(RotationMatrix.MakeZRotation(np.pi),[.5, 0, 0.09])}

builder = pydrake.systems.framework.DiagramBuilder()

station = builder.AddSystem(PandaStation())
station.SetupBinStation()
station.AddManipulandFromFile(
    "drake/examples/manipulation_station/models/061_foam_brick.sdf",
    X_O["initial"])

station.AddManipulandFromFile(
    "drake/examples/manipulation_station/models/061_foam_brick.sdf",
    RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [-.2, -.65, 0.09]))



plant = station.get_multibody_plant()
panda = plant.GetModelInstanceByName("panda")
c1 = AddShape(plant, Cylinder(0.03, 0.5), "c1")
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("c1", c1), RigidTransform([0.15, -0.15, 0.25])) # add a model to test collisions


#c2 = AddShape(plant, Cylinder(0.03, 0.5), "c2")
#body = plant.GetBodyByName("c2")
#plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("c2", c2), RigidTransform([-0.25, -0.5, 0.25])) # add a model to test collisions

#c3 = AddShape(plant, Cylinder(0.5, 0.01), "c3")
#plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("c3", c3), RigidTransform([1, 1, 0.005])) # add a model to test collisions



#bin2 = plant.GetModelInstanceByName("bin2")
#frameC = plant.GetFrameByName("c2")
#frameA = plant.GetFrameByName("c3")
#k0 = np.array([100.,100.,100.])
#d0 = np.array([0.,0.,0.])
#kx = np.array([1000.,1000.,1000.])
#dx = np.array([100.,100.,100.])

#joint = LinearBushingRollPitchYaw(frameA, frameC, k0, d0, kx, dx)
#plant.AddForceElement(joint)

station.Finalize()



# Find the initial pose of the gripper (as set in the default Context)
station_context = station.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(station_context)
plant_ad = plant.ToAutoDiffXd()
print(plant_ad.get_geometry_query_input_port())

#joint.SetForceStiffnessConstants(plant_context, np.array([0,0,0])) # testing if I can change a force element
#joint.SetTorqueStiffnessConstants(plant_context, np.array([0,0,0]))
#print(joint.GetForceStiffnessConstants(plant_context))

X_G = {"initial": plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("panda_hand"))}
X_G, times = make_gripper_frames(X_G, X_O)
print(f"Sanity check: The entire maneuver will take {times['postplace']} seconds to execute.")

# Make the trajectories
traj_p_G = make_gripper_position_trajectory(X_G, times)
traj_R_G = make_gripper_orientation_trajectory(X_G, times)
traj_hand_command = make_hand_command_trajectory(times)

def InterpolatePoseEntry(t):
  return RigidTransform(RotationMatrix(traj_R_G.value(t)), 
                        traj_p_G .value(t))

t_lst = np.linspace(0, times['postplace'], 100)
h = t_lst[1] - t_lst[0]
pose_lst = [InterpolatePoseEntry(tau) for tau in t_lst]

#q_knots = np.array(create_q_knots(pose_lst, h))
q_knots = np.array(create_q_knots(pose_lst, h, panda, plant, plant_context, ['c1']))
q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, q_knots[:, 0:7].T)

q_traj_system = builder.AddSystem(TrajectorySource(q_traj))
#g_traj_system = builder.AddSystem(TrajectorySource(g_traj))

scene_graph = station.get_scene_graph()

meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,
          zmq_url=zmq_url)#, role = Role.kProximity) <- this commented part allows visualization of the collisions

builder.Connect(q_traj_system.get_output_port(),
                  station.GetInputPort("panda_position"))
#builder.Connect(g_traj_system.get_output_port(),
#                  station.GetInputPort("hand_position"))

traj_hand_command = make_hand_command_trajectory(times)
hand_source = builder.AddSystem(pydrake.systems.primitives.TrajectorySource(traj_hand_command))
hand_source.set_name("hand_command")
builder.Connect(hand_source.get_output_port(), station.GetInputPort("hand_position"))

#scene_graph.ToAutoDiffXd()


diagram = builder.Build()


simulator = pydrake.systems.analysis.Simulator(diagram)


t = 0
dt = 0.01
i = 0
meshcat.start_recording()
while t < traj_p_G.end_time():
    simulator.AdvanceTo(t + dt)
    t = t+dt
    i += 1
    #if (i == 1000):
    #    print('testing joint release')
    #    sim_context = simulator.get_mutable_context()
    #    plant_context = plant.GetMyContextFromRoot(sim_context)
    #    joint.SetForceStiffnessConstants(plant_context, np.array([0,0,0])) # testing changing sim context based on certain events
    #    joint.SetTorqueStiffnessConstants(plant_context, np.array([0,0,0]))
    #    joint.SetTorqueDampingConstants(plant_context, np.array([0,0,0]))
    #    joint.SetForceDampingConstants(plant_context, np.array([0,0,0]))
        
meshcat.stop_recording()
meshcat.publish_recording()
'''

'\nX_O = {"initial": RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [-.1, -.65, 0.09]),\n       "goal": RigidTransform(RotationMatrix.MakeZRotation(np.pi),[.5, 0, 0.09])}\n\nbuilder = pydrake.systems.framework.DiagramBuilder()\n\nstation = builder.AddSystem(PandaStation())\nstation.SetupBinStation()\nstation.AddManipulandFromFile(\n    "drake/examples/manipulation_station/models/061_foam_brick.sdf",\n    X_O["initial"])\n\nstation.AddManipulandFromFile(\n    "drake/examples/manipulation_station/models/061_foam_brick.sdf",\n    RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [-.2, -.65, 0.09]))\n\n\n\nplant = station.get_multibody_plant()\npanda = plant.GetModelInstanceByName("panda")\nc1 = AddShape(plant, Cylinder(0.03, 0.5), "c1")\nplant.WeldFrames(plant.world_frame(), plant.GetFrameByName("c1", c1), RigidTransform([0.15, -0.15, 0.25])) # add a model to test collisions\n\n\n#c2 = AddShape(plant, Cylinder(0.03, 0.5), "c2")\n#body = plant.GetBodyByName("c2")\n#plant.Wel

In [13]:
#file = open("media/videos/panda_collisions.html", "w")
#file.write(meshcat.vis.static_html())
#file.close()

# testing Panda IK Traj

In [36]:
builder = pydrake.systems.framework.DiagramBuilder()
station = builder.AddSystem(PandaStation())
station.SetupDefaultStation()

station.Finalize()

plant = station.get_multibody_plant()
panda = plant.GetModelInstanceByName("panda")

station_context = station.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(station_context)


initial = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("panda_hand"))
goal = RigidTransform(RotationMatrix(RollPitchYaw(np.pi, 0, 0)),[-0.3, -0.5, 0.3])

w1 = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("panda_hand"))
traj = Trajectory(w1)
w2 = RigidTransform(RotationMatrix(RollPitchYaw(0, 0, np.pi)),[0.3, -0.5, 0.3])
traj.add_waypoint(w2, 6, 20)
w3 = RigidTransform(RotationMatrix(RollPitchYaw(np.pi, 0, 0)),[-0.3, -0.4, 0.3])
traj.add_waypoint(w3, 12, 20)
w4 = RigidTransform(RotationMatrix(RollPitchYaw(0, np.pi/2, np.pi/2)),[0.4, 0.4, 0.3])
traj.add_waypoint(w4, 20, 20)
w5 = RigidTransform(RotationMatrix(RollPitchYaw(0, 0, np.pi/2)),[0, -0.5, 0.5])
traj.add_waypoint(w5, 27, 20)
w6 = RigidTransform(RotationMatrix(RollPitchYaw(0, 0, np.pi/2)),[0, -0.5, 0.7])
traj.add_waypoint(w6, 32, 20)

traj_maker = PandaIKTraj(plant, plant_context, panda, traj)

q_nominal = np.array([ 0., 0.55, 0., -1.45, 0., 1.58, 0.])
#traj_maker.AddJointCenteringCost(q_nominal)

traj_maker.Solve()
q_traj = traj_maker.get_q_traj()


q_traj_system = builder.AddSystem(TrajectorySource(q_traj)) 
scene_graph = station.get_scene_graph()

meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions

builder.Connect(q_traj_system.get_output_port(),
                  station.GetInputPort("panda_position"))

traj_hand_command = make_hand_command_trajectory(times)
hand_source = builder.AddSystem(pydrake.systems.primitives.TrajectorySource(traj_hand_command))
hand_source.set_name("hand_command")
#builder.Connect(hand_source.get_output_port(), station.GetInputPort("hand_position"))


diagram = builder.Build()




#SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())
simulator = pydrake.systems.analysis.Simulator(diagram)
station_context = station.GetMyContextFromRoot(simulator.get_mutable_context())
station.GetInputPort("hand_position").FixValue(station_context, [0.08]) # taking the desired hand separation

t = 0

meshcat.start_recording()

simulator.AdvanceTo(32)

meshcat.stop_recording()
meshcat.publish_recording()
#SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6006...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7006/static/
Connected to meshcat-server.


In [25]:
#file = open("media/videos/traj_optim_v1.html", "w")
#file.write(meshcat.vis.static_html())
#file.close()

## Making rectangular panda collision geometries

In [45]:
'''
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
Parser(plant, scene_graph).AddModelFromFile("/home/agrobenj/workspace/PandaStation/models/panda_with_box_collision.urdf")
plant.Finalize()


diagram = builder.Build()
context = diagram.CreateDefaultContext()

meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=scene_graph.get_query_output_port(),
          delete_prefix_on_load=True,
          zmq_url=zmq_url)#, role = Role.kProximity)

meshcat.load()
'''

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6006...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7006/static/
Connected to meshcat-server.
